**Instalando o GROQ**

In [2]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.6 MB/s eta 0:00:00


**Definição de uma variavel de ambiente da chave API**

No caso com o nome 'GROQ_API_KEY', junto com a chave que foi adiquirida.



In [3]:
import os
os.environ['GROQ_API_KEY'] = "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"

**Inicialização do cliente GROQ e teste da chave API**

In [4]:
import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

#Iniciamos o groq e aplicamos a chave api

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

#Uma mensagem simples para testar se tudo está certo até agora

print(chat_completion.choices[0].message.content)

Fast language models are crucial in modern natural language processing (NLP) due to their numerous benefits and applications. Here are some key reasons explaining their importance:

1. **Efficient Processing**: Fast language models can process and analyze large amounts of text data quickly, making them ideal for real-time applications such as chatbots, virtual assistants, and language translation software.
2. **Improved User Experience**: Fast language models enable faster response times, which is essential for providing a seamless and engaging user experience. This is particularly important for applications that require immediate feedback, such as voice assistants or live customer support.
3. **Scalability**: Fast language models can handle a large volume of requests and data, making them suitable for large-scale applications such as language translation, sentiment analysis, and text summarization.
4. **Reduced Latency**: Fast language models minimize latency, which is critical in app

**Criação do Agente em si**

In [8]:
class Agent: #Criação da clase agente

  def __init__ (self,client,system): #definição de uma função que irá iniciar meu agente
    self.client = client #Inicializamos primeiro o cliente
    self.system = system #Logo em seguida iremos inicializar o "System Prompt"
    self.messages = [] #Criação de uma lista de um loop de mensagens que o agente irá incluir em sua memória
    if self.system is not None: #Caso exista uma mensagem do sistema nos acidionaremos à nossa lista de mensagens
      self.messages.append({"role": "system", "content": self.system})

  def __call__ (self, message = ""): #Uma chamada que será executada sem que o Agent for executado
    if message: #Se o usuária enviar uma mensagem ela será adicionada na lista
      self.messages.append({"role": "user", "content": message})
    result = self.execute() #Agora executaremos o modulo de linguagem, ou seja o Agent
    self.messages.append({"role": "assistant", "content": result}) #Adicionaremos o resultado do "def execute" à lista
    return result

  def execute(self): #Irá rodar todas as mensagens do usuária, junto ao modelo desejado
    completion = client.chat.completions.create(
        messages= self.messages,
        model="llama-3.3-70b-versatile",
    )
    return completion.choices[0].message.content

**Crição do Prompt do sistema e das ferramente que ele terá a disposição**

In [6]:
#Prompt do sistema
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()



In [7]:
#Ferramentas
#tranfomaremos em um função, as ferramentas que foram mencionadas no prompt (calcular, e pegar a massa dos planetas) , para que assim o Agent possa ter acesso a ela

def calculate(operation) -> float:
  return eval(operation)

def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.867e24
        case _:
            return 0.0




**Inicialização do Agent**

In [10]:
#Iremos iniciar com agente com os parâmetros que criamos anteriormente, e também é aqui que damos um nome ao Agent
neil_tyson = Agent(client, system_prompt)

In [11]:
#Nesse momento iremos inserir a nossa duvida (query) pra que o Agent possa solucionar
result = neil_tyson("What is the mass of Earth times 5?") #Estamos chamando nosso agente e executando a função __call__
print(result) #também veremos qual ação ele irá tomar

Thought: To find the mass of Earth times 5, I first need to find the mass of Earth. 

Action: get_planet_mass: Earth
PAUSE


In [12]:
#Iremos ver o que está "passando pela cabeça" do nosso agente quando nos fizemos a pergunta
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [14]:
observation = get_planet_mass("Earth")
print(f"Observation: {observation}")

Observation: 5.972e+24


In [15]:
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

Thought: Now that I have the mass of Earth, I can multiply it by 5 to get the final result.

Action: calculate: 5.972e24 * 5
PAUSE


In [16]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [17]:
observation = calculate("3.285e23 * 5")
print(observation)

1.6425e+24


In [18]:
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

Thought: I have calculated the mass of Earth times 5, so I now have the answer.

Answer: The mass of Earth times 5 is 2.986e+25


In [19]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

**Crição de um loop para o Agent**

Isso irá acabar com a necessidade de uma pessoa ficar chamando o agente toda vez que desejarmos que ele pense (assim como feito anteriormente), através de um loop ele será capaz de fazer isso por conta própria

In [27]:
import re #Importação da biblioteca "Regular Expressions", pois precisaremos analizar os conteudos de uma string

def agent_loop(max_iterations=10, query: str = ""): #As iteração dirão o maximo de vezes que esse loop de pensamentos do Agent irá ocorrer
  agent = Agent(client, system_prompt) #Inicialização do agente
  tools = ['calculate', 'get_planet_mass'] #Inicializando o nome das ferramentas que temos
  next_prompt = query
  i = 0
  while i < max_iterations: #Inicio do loop
    i += 1
    result = agent(next_prompt) #Chamaremos o Agent e verificaremos o resultado
    print(result)

    if "PAUSE" in result and "Action" in result: #Se o resultado tiver uma ação e pausa
      action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
      chosen_tool = action[0][0]
      arg = action[0][1]

      if chosen_tool in tools:
        result_tool = eval(f"{chosen_tool}('{arg}')")
        next_prompt = f"Observation: {result_tool}"

      else:
        next_prompt = "Observation: Tool not found"

      print(next_prompt)
      continue

    if "Answer" in result: #A resposta foi encontrada e podemos sair do loop
      break




In [29]:
agent_loop(query="What is the mass of Earth plus the mass of Mercury and all of that times 5?")

Thought: I need to find the mass of Earth and the mass of Mercury, then add them together and multiply the result by 5. First, I'll find the mass of Earth.

Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: Now that I have the mass of Earth, I need to find the mass of Mercury.

Action: get_planet_mass: Mercury
PAUSE
Observation: 3.285e+23
Thought: I now have the masses of both Earth and Mercury. Next, I'll add them together.

Action: calculate: 5.972e+24 + 3.285e+23
PAUSE
Observation: 6.300500000000001e+24
Thought: Now that I have the sum of the masses of Earth and Mercury, I need to multiply this result by 5 to get the final answer.

Action: calculate: 6.300500000000001e+24 * 5
PAUSE
Observation: 3.1502500000000004e+25
Thought: I have now calculated the mass of Earth plus the mass of Mercury and multiplied the result by 5, so I have the final answer.

Answer: The mass of Earth plus the mass of Mercury and all of that times 5 is 3.1502500000000004e+25.
